<a href="https://colab.research.google.com/github/kneecat/machine-learning/blob/master/ABC147%20C%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

・**実験目的**
<br>
なにかを実装してみたい（主に機械学習のシステム）時に、今まではネットや本で調べながら作ってきた。しかし、プログラミングの力がないとそもそもこうやって作れるのではないか？という発想自体が浮かばないのでプログラミングにどっぷりと浸かる機会が必要だと思った。問題集をひたすら解くのも良いが、問題を解くこと自体に動機はない（自分で考えた問題ではないため、必要に迫られていないため）ので問題を解くこと自体にモチベーションを得るのではなく問題を解くことで得られる報酬（ここではスコアによってランク付けがされるAtCoderのシステム）からモチベーションを得られるのではないかと思い、教義プログラミングを始めた。
<br>
・**実験方法**
<br>
ABC("AtCoder Beigner Contest")の第147回を解きなおす。
<br>
・**実験結果**
<br>
以下に示す。

**・AtCoderについて**
<br>
競技プログラミングを行う国内最大のサイト
<br>
**・ABCについて**
<br>
* ほぼ毎週行われているコンテスト
* ウェブ上で誰でも簡単に受けることが出来る
* 試験時間は２１：００～２２：４０の１００分間
* ネットや本を見たり、自作の関数を使用することなどは認められている
* 複数人で相談しあって解くことは不正行為として禁止されている
* 試験問題はA～F問題の６つで構成されている（A→B→C→D→E→Fの順でむずかしい）

**問題文**
<br>
1からNまでの番号が振り分けられたN人の人がいる。この人たちは  
①正しいことしか言わない人（正直者）
<br>
②正しいことも言うし間違っていることも言う人（不適切な人） 
<br>
に分けられる。
人iは$A_i$個の証言を持っている。また、その証言のj個目の要素は2つの整数$x_{ij},y_{ij}$で構成されている。（$x$は人物名、$y=1$で正直者、$y=0$で不適切な人。）このN人の中には最大で何人の正直者が存在できるか？ただし
<br>
* 入力はすべて整数
* $1\leqq{N}\leqq{15}$
* $0\leqq{A_i}\leqq{N-1}$
* $1\leqq{x_{ij}}\leqq{N}$
* $x_{ij}\neq{i}$
* $x_{ij1}\neq{x_{ij2}}(j_1\neq{j_2})$
* $y_{ij}=0,1$
<br>
という条件がある。入力は
<br>
<br>
N
<br>
$A_1$
<br>
$x_{11}\quad{y_{11}}$
<br>
・
<br>
・
<br>
・
<br>
$x_{1A_1}\quad{y_{1A_1}}$
<br>
・
<br>
・
<br>
・
<br>
$A_N$
<br>
$x_{N1}\quad{y_{N1}}$
<br>
・
<br>
・
<br>
・
<br>
$x_{NA_N}\quad{y_{NA_N}}$
<br>
<br>
のように与えられる。


In [0]:
#誤答例
N = int(input())
a = [[] for _ in range(N)]
b = []
for i in range(N):
  A_i = int(input())
  b.append(A_i)
  for j in range(A_i):
    a[i].append(list(map(int, input().split())))
    
book = a
max_honest = 0

i = 0
while i < N:
  honest_p = []
  unkind_p = []
  honest_p.append(i)
  contradiction = False
  while True:
    changed = False
    for j in honest_p:
      for k in range(b[j]):
        people = book[j][k][0] - 1
        testimony = book[j][k][1]
        if testimony and people in unkind_p:
          max_honest = 0
          contradiction = True
          break
        if not testimony and people in honest_p:
          max_honest = 0
          contradiction = True
          break
        if testimony and not people in honest_p:
          honest_p.append(people)
          changed = True
        if testimony and not people in unkind_p:
          unkind_p.append(people)
          changed = True
    if contradiction:
      break
    if len(honest_p) > max_honest:
      max_honest = len(honest_p)
    if not changed:
      break
  i += 1

print(max_honest)

↑のやつだと**最初の正直者の仮定**を一人ずつしかおこなっていないので不正解となった

**bit全探索**という方法を用いる

In [0]:
#pythonでの2進数の取り扱い
num = 3
print(num)
print("type=", type(num))
bin_num = bin(num)
print(bin_num)
#str型になる
print("type=", type(bin_num))

3
type= <class 'int'>
0b11
type= <class 'str'>


In [0]:
num = 9
bin_num = bin(num)
print(bin_num)
#以下の不等号で右にシフトできる。その後１との論理和をとる。
print((num >> 3) & 1)

0b1001
1


In [1]:
#正解例
N = int(input())
a = [[] for _ in range(N)]
for i in range(N):
  A_i = int(input())
  for j in range(A_i):
    a[i].append(list(map(int, input().split())))

max_honest = 0
for i in range(2**N):
  contradiction = False
  honest_p = []
  unkind_p = []
  doubt_honest_p = []
  doubt_unkind_p = []
  length = len(format(2**N-1, "b"))
  bin_num = format(i, "b").zfill(length)
  for j in range(length):
    if (i >> j) & 1:
      if not j+1 in honest_p:
      	honest_p.append(j+1)
      if not j+1 in doubt_honest_p:
      	doubt_honest_p.append(j+1)
      for k in a[j]:
        p = k[0]
        t = k[1]
        if t:
          if not p in honest_p:
          	honest_p.append(p)
        else:
          if not p in unkind_p: 
          	unkind_p.append(p)
    else:
      if not j+1 in unkind_p:
        unkind_p.append(j+1)
      if not j+1 in doubt_unkind_p:
      	doubt_unkind_p.append(j+1)
  if len(list(set(honest_p) & set(unkind_p))) != 0:
    contradiction = True
  if not contradiction and max_honest <= len(doubt_honest_p):
    max_honest = len(doubt_honest_p)
    
print(max_honest)

KeyboardInterrupt: ignored

**・まとめ**
<br>
自分の知らないプログラミングの知識が深まるのでやりがいがある。また、実行時間を短くするという考えがなかったので新たな視点が得られた。